# Test the churn analysis in notebook 

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.models import load_model
from pathlib import Path
import os

In [29]:

project_root = Path.cwd().parents[0]
file_path = os.path.join(project_root, "data", "bank_churn.csv")

In [31]:
df = pd.read_csv(file_path)
df.head(5)

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,100001,Smith,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,100002,Johnson,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,100003,Williams,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,100004,Brown,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,100005,Jones,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [43]:
df['HasCrCard'].unique()

array([1, 0])

In [32]:
#Drop dimension which doesn't add value
X = df.drop(['CustomerId', 'Surname', 'Exited'], axis=1)
y = df['Exited']

#Label encoder for categorical columns 
X['Geography'] = LabelEncoder().fit_transform(X['Geography'])
X['Gender']= LabelEncoder().fit_transform(X['Gender'])

In [33]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [41]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      26 non-null     int64  
 1   Geography        26 non-null     int64  
 2   Gender           26 non-null     int64  
 3   Age              26 non-null     int64  
 4   Tenure           26 non-null     int64  
 5   Balance          26 non-null     float64
 6   NumOfProducts    26 non-null     int64  
 7   HasCrCard        26 non-null     int64  
 8   IsActiveMember   26 non-null     int64  
 9   EstimatedSalary  26 non-null     float64
dtypes: float64(2), int64(8)
memory usage: 2.2 KB


In [34]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [35]:
X_train, X_test , y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, random_state=20)

In [ ]:
X_train.shape[1]

20

In [36]:
model = Sequential([
    Dense(16,activation='relu', input_shape=(X_train.shape[1],)), 
    Dense(8, activation = 'relu'),
    Dense(1,activation='sigmoid')
])

d:\Python\Eron Learning\GenerativeAI\Project1-CustomerChurn\.venv\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train,y_train, epochs=30, batch_size=16, validation_split=0.1, verbose=1)